# kakao 주소 검색 API를 활용하여 X_Y 좌표 수집

- [좌표X,좌표Y 모두 맵핑](../data/unique_apt_group_xy.csv)
- [아파트 중복제거 좌표X,좌표Y 결측치 있음](../data/unique_apt_group.csv)

In [2]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)
import module

In [3]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# AppleGothic
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
df = module.file_load.unique_apt_group_file()

In [4]:
(x,y) = module.api.get_coordinates('서울특별시 송파구 올림픽로 300')
print(x,y)

127.104301829165 37.5137129859207


In [5]:
none_list = filtered_nan_rows = df[df['좌표X'].isna()]

In [6]:
df.loc[758]

아파트_unique_key    강남구_개포동_189_개포자이프레지던스_2023
시                                      서울특별시
구                                        강남구
동                                        개포동
시군구                            서울특별시 강남구 개포동
번지                                       189
본번                                     189.0
아파트명                               개포자이프레지던스
건축년도                                    2023
도로명                                   삼성로 14
좌표Y                                37.485699
좌표X                               127.069985
Name: 758, dtype: object

In [7]:
for index, row in none_list.iterrows():
    try:
        query = module.api.search_apt_query(row)
        (x,y) = module.api.get_coordinates(query)
        if x != None:
            df.loc[index, '좌표X'] = x
            df.loc[index, '좌표Y'] = y
        else:
            print(index, row['아파트명'], query)
    except:
        print('err', index, row['아파트명'], query)
        continue

주소를 찾을 수 없습니다. 서울특별시 강남구 개포동 138
762 디에이치아너힐즈 서울특별시 강남구 개포동 138
주소를 찾을 수 없습니다. 서울특별시 강서구 방화동 888
1092 마곡경남아너스빌 서울특별시 강서구 방화동 888
주소를 찾을 수 없습니다. 서울특별시 강서구 방화동 897
1095 마곡일성트루엘 서울특별시 강서구 방화동 897
주소를 찾을 수 없습니다. 서울특별시 강서구 화곡동 1027-50
1165 우장산숲아이파크 서울특별시 강서구 화곡동 1027-50
주소를 찾을 수 없습니다. 서울특별시 관악구 봉천동 1152
1198 관악푸르지오102동 서울특별시 관악구 봉천동 1152
주소를 찾을 수 없습니다. 서울특별시 관악구 신림동 1644
1242 힐스테이트뉴포레 서울특별시 관악구 신림동 1644
주소를 찾을 수 없습니다. 서울특별시 구로구 고척동 97
1323 더헤리츠 서울특별시 구로구 고척동 97
주소를 찾을 수 없습니다. 서울특별시 구로구 항동 가-238
1411 하버라인4단지 서울특별시 구로구 항동 가-238
주소를 찾을 수 없습니다. 서울특별시 서대문구 북아현동 149
1910 e편한세상신촌2단지 서울특별시 서대문구 북아현동 149
주소를 찾을 수 없습니다. 서울특별시 서대문구 북아현동 149
1911 e편한세상신촌3단지 서울특별시 서대문구 북아현동 149
주소를 찾을 수 없습니다. 서울특별시 서대문구 홍제동 270-1
1962 홍제역해링턴플레이스 서울특별시 서대문구 홍제동 270-1
주소를 찾을 수 없습니다. 서울특별시 서초구 잠원동 52-2
2054 르엘신반포파크애비뉴 서울특별시 서초구 잠원동 52-2
주소를 찾을 수 없습니다. 서울특별시 성동구 금호동1가 280
2071 이편한세상금호파크힐스 서울특별시 성동구 금호동1가 280
주소를 찾을 수 없습니다. 서울특별시 성북구 장위동 173-114
2180 래미안장위퍼스트하이 서울특별시 성북구 장위동 173-114
주소를 찾을 수 없습니다. 서울특별시 중랑구 상봉동 785
2770 건영2차

In [16]:
unique = pd.read_csv('unique_apt_group_xy.csv')

In [17]:
xy_df = pd.read_csv('../data/x_y_key_map.csv')

In [25]:
unique[unique['좌표X'].isna()].head(1)

,아파트_unique_key,시,구,동,시군구,번지,본번,아파트명,건축년도,도로명,좌표Y,좌표X
762,강남구_개포동_138_디에이치아너힐즈_2019,서울특별시,강남구,개포동,서울특별시 강남구 개포동,138,138.0,디에이치아너힐즈,2019,삼성로 11,NaN,NaN


In [22]:
xy_df.head(1)

,key,search_key,좌표X,좌표Y,아파트명,search_key1,search_key2,구,동,시,번지,본번,부번,도로명
0,강남구개포동개포로 264개포래미안포레스트,개포로 264 개포래미안포레스트,127.054031,37.47848,개포래미안포레스트,서울특별시 강남구 개포동 개포로 264,서울특별시 강남구 개포동 1282,강남구,개포동,서울특별시,1282,1282.0,0.0,개포로 264


## 결측치 보완

In [28]:
for index, row in unique[unique['좌표X'].isna()].iterrows():
    key_query = row['시군구'] + ' ' + row['도로명']
    try:
        x = xy_df[xy_df['search_key1'] == key_query]['좌표X'].values[0]
        y = xy_df[xy_df['search_key1'] == key_query]['좌표Y'].values[0]
        unique.loc[index, '좌표X'] = x
        unique.loc[index, '좌표Y'] = y
    except:
        print('err', index, row['아파트명'])
        continue

err 7586 문화
err 7612 반도1차
err 8109 래미안라클래시
err 8731 진달래2차
err 8737 태양
err 8738 마포지구시범6동
err 8739 삼호가든맨션
err 8743 진달래3차
err 8746 금호
err 8748 신용산빌딩


In [29]:
# unique.to_csv('../data/unique_apt_group_xy.csv', index=False)